In [41]:
import tensorflow as tf
import numpy as np

In [42]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]

The same implementation with keras

In [ ]:
from tensorflow import keras

class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation
        
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(
            shape=(input_dim, self.units),
            initializer='random_normal'
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer='zeros'
        )
    
    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [43]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights
    

In [44]:
model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.relu),
])
assert len(model.weights) == 4

In [45]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = int(math.ceil(len(self.images) / self.batch_size))
    
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [46]:
def update_weights_by_hand(gradients, weights, learning_rate=1e-3):
    for g, w in zip(gradients,weights):
        w.assign_sub(learning_rate * g)
        
# as tensor optimiser
from tensorflow.keras import optimizers
optimizer = optimizers.SGD(learning_rate=1e-3)
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [47]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [48]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at {batch_counter}: {loss:.2f}")

In [49]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at 0: 4.49
loss at 100: 2.26
loss at 200: 2.25
loss at 300: 2.11
loss at 400: 2.23
Epoch 1
loss at 0: 1.93
loss at 100: 1.89
loss at 200: 1.85
loss at 300: 1.72
loss at 400: 1.83
Epoch 2
loss at 0: 1.61
loss at 100: 1.59
loss at 200: 1.52
loss at 300: 1.43
loss at 400: 1.51
Epoch 3
loss at 0: 1.34
loss at 100: 1.35
loss at 200: 1.25
loss at 300: 1.21
loss at 400: 1.27
Epoch 4
loss at 0: 1.14
loss at 100: 1.17
loss at 200: 1.05
loss at 300: 1.05
loss at 400: 1.11
Epoch 5
loss at 0: 1.00
loss at 100: 1.03
loss at 200: 0.91
loss at 300: 0.93
loss at 400: 0.99
Epoch 6
loss at 0: 0.89
loss at 100: 0.92
loss at 200: 0.81
loss at 300: 0.84
loss at 400: 0.90
Epoch 7
loss at 0: 0.81
loss at 100: 0.84
loss at 200: 0.73
loss at 300: 0.77
loss at 400: 0.83
Epoch 8
loss at 0: 0.74
loss at 100: 0.77
loss at 200: 0.67
loss at 300: 0.71
loss at 400: 0.78
Epoch 9
loss at 0: 0.69
loss at 100: 0.71
loss at 200: 0.62
loss at 300: 0.67
loss at 400: 0.74


In [50]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"Accuracy: {np.sum(matches) / len(test_images)}")
print(f"accuracy: {matches.mean():.2f}")

Accuracy: 0.812
accuracy: 0.81
